## Optimización de hiper-parámetros

### Recapitulando

In [1]:
import pandas as pd
#import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn import tree

from sklearn.tree import DecisionTreeClassifier

#from IPython.display import Image  

In [2]:
directorio_data = '../data/'

x_training_file = directorio_data + 'x_traininig.csv'
y_training_file = directorio_data + 'y_traininig.csv'

x_testing_file = directorio_data + 'x_testing.csv'
y_testing_file = directorio_data + 'y_testing.csv'

In [3]:
df_x_training_validacion = pd.read_csv(x_training_file)
df_y_training_validacion = pd.read_csv(y_training_file)

X_testing = pd.read_csv(x_testing_file)
Y_testing = pd.read_csv(y_testing_file)

### Separamos nuestro dataset training-validación en training y validación

In [4]:
random_state = 661
test_size = 0.3

X_training, X_validacion, Y_training, Y_validacion = train_test_split(
    df_x_training_validacion, 
    df_y_training_validacion, 
    test_size=test_size, 
    random_state=random_state,
    stratify=df_y_training_validacion
)

### Dejamos solo las columnas que queremos

In [5]:
columnas = [
    "review_overall",
    "review_aroma",
    "review_appearance",
    "review_palate",
    "review_taste",
    "beer_abv"
]

In [6]:
X_training = X_training[columnas]
X_validacion = X_validacion[columnas]
X_testing = X_testing[columnas]

## Optimización con grid search

In [35]:
punto_de_corte = 0.11111111

max_depth_grid_search = [1,2,3,4,5,6,7,8,9,10,11,12]

for max_depth in max_depth_grid_search:
    criterion='gini'
    clf = DecisionTreeClassifier(
                criterion = criterion,
                max_depth=max_depth,
                random_state=random_state
            )
    _ = clf.fit(X_training, Y_training)
    Y_validacion_pred_proba = clf.predict_proba(X_validacion)
    Y_validacion_pred_proba = pd.DataFrame(Y_validacion_pred_proba, columns = ['NO_prob', 'SI_prob'])
    Y_validacion_pred_proba['prediccion'] = 'NO'
    Y_validacion_pred_proba.loc[Y_validacion_pred_proba['SI_prob']>punto_de_corte,'prediccion'] = 'SI'
    cm = confusion_matrix(Y_validacion, Y_validacion_pred_proba['prediccion'], labels=['SI', 'NO'])
    true_positives = cm[0,0]
    true_negatives = cm[1,1]
    false_positives = cm[1, 0]
    false_negatives = cm[0,1]
    metrica_negocio = ((4000*true_positives) - (500*false_positives))/(true_positives + false_negatives)
    print(max_depth, metrica_negocio)
    

1 0.0
2 0.0
3 16.736401673640167
4 43.93305439330544
5 334.72803347280336
6 345.1882845188284
7 207.11297071129707
8 309.6234309623431
9 303.34728033472805
10 372.3849372384937
11 364.01673640167365
12 309.6234309623431


### Probamos con testing

In [40]:
directorio_data = '../data/'

x_training_file = directorio_data + 'x_traininig.csv'
y_training_file = directorio_data + 'y_traininig.csv'

x_testing_file = directorio_data + 'x_testing.csv'
y_testing_file = directorio_data + 'y_testing.csv'

In [41]:
df_x_training_validacion = pd.read_csv(x_training_file)
df_y_training_validacion = pd.read_csv(y_training_file)

X_testing = pd.read_csv(x_testing_file)
Y_testing = pd.read_csv(y_testing_file)

In [42]:
columnas = [
    "review_overall",
    "review_aroma",
    "review_appearance",
    "review_palate",
    "review_taste",
    "beer_abv"
]

In [43]:
X_training_validacion = df_x_training_validacion[columnas]
X_testing = X_testing[columnas]

In [47]:
max_depth = 6
criterion='gini'
clf = DecisionTreeClassifier(
            criterion = criterion,
            max_depth=max_depth,
            random_state=random_state
        )
_ = clf.fit(X_training_validacion, df_y_training_validacion)

In [48]:
umbral_de_corte = 0.11111111

Y_testing_pred_proba = clf.predict_proba(X_testing)
Y_testing_pred_proba = pd.DataFrame(Y_testing_pred_proba, columns = ['NO_prob', 'SI_prob'])

Y_testing_pred_proba['prediccion'] = 'NO'
Y_testing_pred_proba.loc[Y_testing_pred_proba['SI_prob']>umbral_de_corte,'prediccion'] = 'SI'

cm = confusion_matrix(Y_testing, Y_testing_pred_proba['prediccion'], labels=['SI', 'NO'])
true_positives = cm[0,0]
true_negatives = cm[1,1]
false_positives = cm[1, 0]
false_negatives = cm[0,1]

metrica_negocio = ((4000*true_positives) - (500*false_positives))/(true_positives + false_negatives)
metrica_negocio

365.0

### Optimización con Grid search: max_depth + min_samples_split